In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
'''
The hardest thing about this part was to find out about existence of SBERT :D
'''

C:\PodRec\main\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


'\nThe hardest thing about this part was to find out about existence of SBERT :D\n'

In [2]:
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
model = SentenceTransformer('all-MiniLM-L6-v2')
df = pd.read_csv('../datasets/podcasts.csv')
emb = np.load('../datasets/sbert_embeddings.npy')

In [4]:
text = "podcast about scientifical approach to health and biohacking"

In [7]:
res = model.encode(text)

In [8]:
res.shape

(384,)

In [9]:
example = df['description'][0]


In [10]:
emb[0].shape

(384,)

In [11]:
cosine_similarity(res.reshape(1,-1),emb[0].reshape(1,-1))[0][0] #ok 

0.2840082

In [12]:
cosine_similarity_user = []
for index,embedding in enumerate(emb):
    cosine_similarity_user.append((index,cosine_similarity(res.reshape(1,-1),embedding.reshape(1,-1))[0][0]))

In [13]:
cosine_similariry_user = sorted(cosine_similarity_user,key=lambda x:x[1],reverse=True)

In [14]:
cosine_similariry_user[:10]

[(12397, 0.73682666),
 (8678, 0.7367557),
 (5387, 0.7330305),
 (4028, 0.7224921),
 (6451, 0.7043869),
 (4992, 0.7002674),
 (6607, 0.6950602),
 (12959, 0.6894901),
 (2131, 0.6878042),
 (7014, 0.6876585)]

In [15]:
cosine_similarity_matrix = cosine_similarity(emb)

In [16]:
np.save('cos_sim_sbert.npy',cosine_similarity_matrix)

In [17]:
# function, that i later copypasted to quering_functions in 'main', not important
def get_sbert_top(dataset,itunes_id,sbert_cos_sim,top=10):
    row = dataset[dataset['itunes_id'] == itunes_id]
    if not row.empty:
        row = row.index[0]
    else:
        row = None
        return None

    indexed_embeddings = set((i,sbert_cos_sim[row][i]) for i in range(len(sbert_cos_sim[row])))
    sorted_emb = sorted(indexed_embeddings,key=lambda item:item[1],reverse=True)
    print("sort_emb first 10",sorted_emb[:10])
    print("sort_emb last 10",sorted_emb[-10:])
    sorted_emb = sorted_emb[:top]
    indicies = [sorted_emb[i][0] for i  in range(len(sorted_emb))]
    return dataset.iloc[indicies,:]

In [18]:
get_sbert_top(df,1705752628	,cosine_similarity_matrix,10)

sort_emb first 10 [(14199, 1.0000001), (10556, 0.5546344), (10483, 0.5343531), (7723, 0.5286615), (10279, 0.51388395), (5283, 0.51190776), (10055, 0.50965255), (6329, 0.50441176), (12340, 0.502966), (10318, 0.49014422)]
sort_emb last 10 [(8909, -0.10837166), (9828, -0.1097804), (5890, -0.110543095), (8198, -0.1114791), (9322, -0.11215882), (558, -0.11721636), (12302, -0.11826211), (4556, -0.123417065), (667, -0.124815255), (5999, -0.13890886)]


,Unnamed: 0,podcast_name,podtail_link,itunes_id,podcastindex_id,description,rss_link,language,categories,number_of_podcasts,rating,number_of_reviews,index_description
14199,14199,How To Buy a House,https://podtail.com/podcast/how-to-buy-a-house-1/,1705752628,6599180,"A simple ""How To"" Guide on how to buy a house ...",https://www.spreaker.com/show/5946861/episodes...,en,"Education,Business,Investing,Entrepreneurship,...",1.0,0.0,0.0,"[5, 837, 105, 3, 1614, 9, 105, 3, 1508, 5, 422..."
10556,10556,The Real Estate Sales Podcast,https://podtail.com/podcast/the-real-estate-sa...,1534461762,2136156,The Real Estate Sales Podcast is designed to p...,https://therealestatesalespodcast.libsyn.com/rss,en,Business,100.0,4.9,72.0,"[0, 406, 2668, 938, 9868, 7, 1577, 3, 354, 158..."
10483,10483,Financial Survival Network,https://podtail.com/podcast/financial-survival...,485166512,848654,A show about how to thrive in the New Economy....,https://www.spreaker.com/show/3244966/episodes...,en,Business,1000.0,4.7,118.0,"[5, 194, 31, 105, 3, 11488, 4, 0, 86, 671, 12,..."
7723,7723,The Nick Huber Show,https://podtail.com/podcast/the-nick-huber-show/,1576120606,4084483,The Nick Huber show is a short-form podcast on...,https://nickhubershow.libsyn.com/rss,en,Business,100.0,4.7,50.0,"[0, 194, 7, 5, 737, 302, 9868, 9, 406, 2668, 1..."
10279,10279,Creating Wealth Simplified,https://podtail.com/podcast/creating-wealth-si...,1442697566,337112,"Welcome to Creating Wealth Simplified, a podca...",https://www.podetize.com/good-deeds-note-inves...,en-us,"Business,Investing,Careers,Entrepreneurship",258.0,5.0,89.0,"[315, 3, 1107, 2365, 8516, 5, 9868, 6, 5370, 2..."
5283,5283,The Evernest Real Estate Investor,https://podtail.com/podcast/the-evernest-real-...,1589852515,4403924,Are you a new investor looking to buy your fir...,https://evernestrealestateinvestor.libsyn.com/rss,en,"Business,Investing,Education,How To",100.0,4.8,17.0,"[21, 20, 5, 86, 6280, 1006, 3, 1508, 38, 88, 8..."
10055,10055,How to Sell,https://podtail.com/podcast/sales-iq-podcast/,1444853887,1009054,How to Sell brings you weekly interviews with ...,https://feeds.captivate.fm/salesiqpodcast/,en,"Business,Entrepreneurship,Marketing",223.0,4.8,85.0,"[105, 3, 2051, 2883, 20, 3845, 2944, 10, 0, 10..."
6329,6329,Jason Hartman's Quick Start Podcast,https://podtail.com/podcast/jason-hartman-s-qu...,1441016571,238397,Find all the episodes from Jason Hartman that ...,https://quick.libsyn.com/rss,en,"Business,Investing,Education",66.0,4.9,13.0,"[254, 26, 0, 2348, 16, 6, 39, 3684, 20, 0, 210..."
12340,12340,Your Home,https://podtail.com/podcast/your-home/,1515183398,451833,We delve into the Real Estate and Auction mark...,https://rss.whooshkaa.com/rss/podcast/id/12592,en-us,"News,Business",43.0,0.0,0.0,"[71, 21517, 69, 0, 406, 2668, 1, 5398, 367, 8,..."
10318,10318,Commercial Real Estate Investing From A-Z,https://podtail.com/podcast/commercial-real-es...,1451874700,569814,Getting started with Commercial Real Estate In...,https://anchor.fm/s/558aa20/podcast/rss,en,"Business,Investing,Education,How To",177.0,4.9,142.0,"[480, 334, 10, 1332, 406, 2668, 6940, 13, 33, ..."


In [19]:
df[df['podcast_name'].str.contains("Dateline NBC")] # better that starting with for finding podcast by user

,Unnamed: 0,podcast_name,podtail_link,itunes_id,podcastindex_id,description,rss_link,language,categories,number_of_podcasts,rating,number_of_reviews,index_description
0,0,Dateline NBC,https://podtail.com/podcast/dateline-nbc/,1464919521,945141,"Current and classic episodes, featuring compel...",https://podcastfeeds.nbcnews.com/dateline-nbc,en,"News,Society,Culture,True Crime",469.0,4.4,34700.0,"[343, 1, 2989, 2348, 2421, 5975, 506, 1256, 11..."
